In [7]:
import yaml
import openai
from pprint import pprint
import json

In [8]:
with open("configs/TJ_OPENAI_KEY.yaml", "r") as f:
    cx = yaml.safe_load(f)

In [9]:
client = openai.OpenAI(
    # This is the default and can be omitted
    api_key=cx['openai_api_key'],
)

### Step 1: Ask for Company, Department and Designation

In [24]:
company = "Financial Technology SaaS company"
designation = "Engineering Manager"
department = "Technology team"

### Step 2: Get 10 Questions for the Designation/Department/Company

In [37]:
prompt = "What should be the 10 questions that you can ask to find out the skills that a {designation} in {department} has who is working in a {company}?".format(designation=designation, department=department, company=company)
print(prompt)

What should be the 10 questions that you can ask to find out the skills that a Engineering Manager in Technology team has who is working in a Financial Technology SaaS company?


In [28]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-3.5-turbo",
)

In [35]:
print(chat_completion.choices[0].message.to_dict()['content'])

1. Can you describe your experience managing engineering teams within a financial technology SaaS company?
2. How do you approach prioritizing and allocating resources for engineering projects in a fast-paced environment?
3. Can you discuss a successful project that you led and the specific engineering skills that were critical to its success?
4. How do you stay current with new technologies and trends in the financial technology industry?
5. How do you assess and develop the technical skills of your team members?
6. Have you had experience working with data security and compliance regulations specific to financial technology platforms?
7. How do you approach problem-solving and decision-making within your engineering team?
8. Can you discuss a challenging situation you faced as an engineering manager and how you navigated it?
9. How do you foster collaboration and communication among cross-functional teams within the organization?
10. How do you measure the success and performance of 

### Step 3: Get Required Skills that the Designation/Department/Company

In [38]:
prompt2 = "What should be skills that a {designation} in {department} should have who is working in a {company}? Please provide the category and skills inside that category".format(designation=designation, department=department, company=company)
print(prompt2)

What should be skills that a Engineering Manager in Technology team should have who is working in a Financial Technology SaaS company? Please provide the category and skills inside that category


In [39]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt2,
        }
    ],
    model="gpt-3.5-turbo",
)

In [40]:
print(chat_completion.choices[0].message.to_dict()['content'])

Technical Skills:
1. Proficiency in programming languages such as Java, Python, or C++
2. Knowledge of cloud computing technologies such as AWS or Azure
3. Experience with databases such as SQL or NoSQL
4. Familiarity with DevOps tools and practices
5. Understanding of cybersecurity best practices

Managerial Skills:
1. Leadership and team management
2. Strategic planning and decision-making
3. Effective communication and collaboration
4. Ability to prioritize and delegate tasks
5. Budgeting and resource management

Domain Knowledge:
1. Understanding of financial technology (FinTech) industry trends and regulations
2. Knowledge of SaaS business models and subscription-based services
3. Experience working with financial data and APIs
4. Familiarity with payment processing systems and protocols
5. Awareness of compliance and security standards in the financial industry.


### Step 4: Create instructions (prompt) for the assistant usint the generated questions

#### Main Context for the Assistant
```
## Aim: 
- Ask questions to gather information about the skills of the user  
- Use the provided questions as a guideline to ask questions

## Persona: 
- Always stay on point and do not get distracted
- Encourage user to think deeply and articulate their thoughts clearly
- Be encouraging and inquisitive
- Maintain professionalism in all circumstances
- Ensure the user feels supported throughout the conversation

## Restrictions
- Do not deviate from a professional demeanor
- Do not use slang, jargon, or inappropriate language under any circumstances
- Do not provide answers or opinions on the topics discussed
- Do not disclose or discuss any specific reasons behind asking the questions

## Ending the conversation
- End after all information is gathered
- End with this exact phrase "Thank you for your anwsers. Hope you have a good day. Bye."

```

#### Additional Context for each run
```
## Questions to use as guidelines
{{ Add the generated questions }}
```

### Step 5A: Start a conversation using the generated questions

- Create a `thread()` 
- Add a `messsage( { "role": "user", "content": "Let's start the interview" })`  to the `thread()`
- Generate a response from the assistant using `run(<thread_id>)`
    - Give the contructed prompt with the generated questions in `additional_context` parameter
- Add the assistant's response as `messsage( { "role": "assistant", "content": <response> })`  to the `thread()`

### Step 5B: Continue the conversation with the user
- Send the assistant's `response` to the user
- Add the user's response as a `messsage( { "role": "user", "content": <user response> })` to the `thread()`
- Generate a response from the assistant using `run(<thread_id>)` using the same `additional_context`
- Add the assistant's response as `messsage( { "role": "assistant", "content": <response> })`  to the `thread()`

### Step 5C: Lookout for the ending phrase to end the conversation

```
Thank you for your anwsers. Hope you have a good day. Bye.
```

### Step 6: Analyse the Skills from the conversation

```
## Aim: Based on the conversation, 
- Evaluate the user's skills
- For each of the mentioned skills, give a rating as mentioned
    
## Output
For each required skill, provide the following information
- Skill Present: Does conversation mention about the skill
    - Value should be "Yes" if the skill is mentioned in the conversation
    - Value should be "No" if the skill is not mentioned
- Level of knowledge of the skill: How well the user demonstrates this skill
    - Good: Demonstrates exceptional abilities about the skill
    - Medium: User demonstrates some abilities and willingness to learn
    - Low: User lacks enough knowledge of the skill
    - Not present: There is no mention of the skill in the conversation

## Skills
{{ Generated Skills }}

## Conversation
{{ Conversation messages }}
```

### Code Samples

In [19]:
all_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)

In [20]:
for _asst in all_assistants.to_dict().get('data', []):
    print('{}\t{}\t{}'.format(_asst.get('id'), _asst.get('model'), _asst.get('name')))

asst_785xQFlHBDyI84T5aSjmDi98	gpt-3.5-turbo	Test Assistant
asst_JK6xg3nNQZsy3IVTa4rf4SpF	gpt-3.5-turbo-0125	TheInterviewer
asst_tYcT1yCAtsF7cmIWPs4vvrQ4	gpt-4-turbo	


### Assistant Details 
- Name = `Test Assistant`
- ID = `asst_785xQFlHBDyI84T5aSjmDi98`
- Model = `gpt-3.5-turbo`

In [21]:
my_assistant = client.beta.assistants.retrieve("asst_785xQFlHBDyI84T5aSjmDi98")
print(my_assistant.to_json())

{
  "id": "asst_785xQFlHBDyI84T5aSjmDi98",
  "created_at": 1715681747,
  "description": null,
  "instructions": "You are a helpful assistant. Given a set of instructions you will repeat the instructions again.",
  "metadata": {},
  "model": "gpt-3.5-turbo",
  "name": "Test Assistant",
  "object": "assistant",
  "tools": [],
  "response_format": "auto",
  "temperature": 1.0,
  "tool_resources": {},
  "top_p": 1.0
}


In [12]:
assistant = client.beta.assistants.create(
    name="Test Assistant",
    instructions="You are a helpful assistant. Given a set of instructions you will repeat the instructions again.",
    model="gpt-3.5-turbo"
)

In [13]:
assistant

Assistant(id='asst_785xQFlHBDyI84T5aSjmDi98', created_at=1715681747, description=None, instructions='You are a helpful assistant. Given a set of instructions you will repeat the instructions again.', metadata={}, model='gpt-3.5-turbo', name='Test Assistant', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)

In [15]:
from pprint import pprint

pprint(assistant)

Assistant(id='asst_785xQFlHBDyI84T5aSjmDi98', created_at=1715681747, description=None, instructions='You are a helpful assistant. Given a set of instructions you will repeat the instructions again.', metadata={}, model='gpt-3.5-turbo', name='Test Assistant', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)


In [16]:
empty_thread = client.beta.threads.create()
print(empty_thread)

Thread(id='thread_StaAt5VpjJl34IJNF4xjoT62', created_at=1715685440, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [17]:
client.beta.threads.retrieve("thread_StaAt5VpjJl34IJNF4xjoT62")

Thread(id='thread_StaAt5VpjJl34IJNF4xjoT62', created_at=1715685440, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None))

In [19]:
thread_message = client.beta.threads.messages.create(
  "thread_StaAt5VpjJl34IJNF4xjoT62",
  role="user",
  content="How does AI work? Explain it in simple terms.",
)
print(thread_message)

Message(id='msg_YxBaHu6mHKSeQKWygfDdy3Fr', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='How does AI work? Explain it in simple terms.'), type='text')], created_at=1715688238, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_StaAt5VpjJl34IJNF4xjoT62')


In [25]:
json.loads(client.beta.threads.retrieve("thread_StaAt5VpjJl34IJNF4xjoT62").to_json())

{'id': 'thread_StaAt5VpjJl34IJNF4xjoT62',
 'created_at': 1715685440,
 'metadata': {},
 'object': 'thread',
 'tool_resources': {'code_interpreter': {'file_ids': []}}}

In [24]:
import json
json.loads(client.beta.threads.messages.list("thread_StaAt5VpjJl34IJNF4xjoT62").to_json())

{'data': [{'id': 'msg_YxBaHu6mHKSeQKWygfDdy3Fr',
   'assistant_id': None,
   'attachments': [],
   'content': [{'text': {'annotations': [],
      'value': 'How does AI work? Explain it in simple terms.'},
     'type': 'text'}],
   'created_at': 1715688238,
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
   'run_id': None,
   'thread_id': 'thread_StaAt5VpjJl34IJNF4xjoT62'}],
 'object': 'list',
 'first_id': 'msg_YxBaHu6mHKSeQKWygfDdy3Fr',
 'last_id': 'msg_YxBaHu6mHKSeQKWygfDdy3Fr',
 'has_more': False}